In [1]:
import pandas as pd
import numpy as np


In [8]:
import pickle
pickle.format_version


'4.0'

In [2]:
train_data = pd.read_excel(r'Data_Train.xlsx')

In [4]:
train_data['Source'].unique()

array(['Banglore', 'Kolkata', 'Delhi', 'Chennai', 'Mumbai'], dtype=object)

In [4]:
train_data['Destination'].unique()

array(['New Delhi', 'Banglore', 'Cochin', 'Kolkata', 'Delhi', 'Hyderabad'],
      dtype=object)

In [3]:
train_data['Airline'].unique()

array(['IndiGo', 'Air India', 'Jet Airways', 'SpiceJet',
       'Multiple carriers', 'GoAir', 'Vistara', 'Air Asia',
       'Vistara Premium economy', 'Jet Airways Business',
       'Multiple carriers Premium economy', 'Trujet'], dtype=object)

In [7]:
[1,2,3].remove(3)

In [5]:
train_data.head()

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
0,IndiGo,24/03/2019,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897
1,Air India,1/05/2019,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7662
2,Jet Airways,9/06/2019,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,No info,13882
3,IndiGo,12/05/2019,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,5h 25m,1 stop,No info,6218
4,IndiGo,01/03/2019,Banglore,New Delhi,BLR → NAG → DEL,16:50,21:35,4h 45m,1 stop,No info,13302


In [6]:
train_data.drop(["Route", "Additional_Info"], axis = 1, inplace = True)


In [7]:
train_data.replace({"non-stop": 0, "1 stop": 1, "2 stops": 2, "3 stops": 3, "4 stops": 4}, inplace = True)


In [8]:
def minutes(x):
    if len(x)==1:
        if x[0][-1]=='h':
            return int(x[0][:-1])*60
        else:
            return int(x[0][:-1])
    else:
        return int(x[0][:-1])*60+int(x[1][:-1])


In [9]:
train_data['Dep_hr'] = train_data['Dep_Time'].str[0:2]
train_data['Arrival_hr'] = train_data['Arrival_Time'].str[0:2]
train_data['Duration_time'] = train_data['Duration'].str.split().apply(minutes)

In [10]:
train_data['Dep_hr']=train_data['Dep_hr'].astype(int)
train_data['Arrival_hr'] = train_data['Arrival_hr'].astype(int)

In [11]:
Airline = train_data[["Airline"]]

Airline = pd.get_dummies(Airline, drop_first= True)

Source = train_data[["Source"]]

Source = pd.get_dummies(Source, drop_first= True)

Destination = train_data[["Destination"]]

Destination = pd.get_dummies(Destination, drop_first = True)

data_train = pd.concat([train_data, Airline, Source, Destination], axis = 1)


In [12]:
data_train.drop(["Airline", "Source", "Destination"], axis = 1, inplace = True)

In [13]:
data_train.dropna(axis=0,inplace=True)

In [14]:
X = data_train.drop(columns=['Price','Dep_Time','Arrival_Time','Duration','Date_of_Journey'])
y = data_train[['Price']]


In [19]:
X.iloc[:2,:18]

,Total_Stops,Dep_hr,Arrival_hr,Duration_time,Airline_Air India,Airline_GoAir,Airline_IndiGo,Airline_Jet Airways,Airline_Jet Airways Business,Airline_Multiple carriers,Airline_Multiple carriers Premium economy,Airline_SpiceJet,Airline_Trujet,Airline_Vistara,Airline_Vistara Premium economy,Source_Chennai,Source_Delhi,Source_Kolkata
0,0.0,22,1,170,0,0,1,0,0,0,0,0,0,0,0,0,0,0
1,2.0,5,13,445,1,0,0,0,0,0,0,0,0,0,0,0,0,1


In [58]:
X['Total_Stops'].describe()

count    10682.000000
mean         0.824190
std          0.675229
min          0.000000
25%          0.000000
50%          1.000000
75%          1.000000
max          4.000000
Name: Total_Stops, dtype: float64

In [59]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [60]:
from sklearn.ensemble import RandomForestRegressor
reg_rf = RandomForestRegressor()
reg_rf.fit(X_train, y_train)

C:\Users\vemul\AppData\Local\Temp/ipykernel_9944/2751168992.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  reg_rf.fit(X_train, y_train)


RandomForestRegressor()

In [61]:
y_pred = reg_rf.predict(X_test)


In [62]:
reg_rf.score(X_train, y_train)

0.7856981860398503

In [63]:
reg_rf.score(X_test, y_test)

0.6012394767496894

In [65]:
from sklearn import metrics


metrics.r2_score(y_test, y_pred)

0.6012394767496894

In [66]:
import pickle
# open a file, where you ant to store the data
file = open('flight_rf.pkl', 'wb')

# dump information to that file
pickle.dump(reg_rf, file)